In [1]:
!pip install wandb -qqq

In [2]:
hi = 'ResNet34-epochs-10'

In [3]:
!pip install wandb -qqq
!python train_model_wandb.py --layers=34 --epochs=10

Files already downloaded and verified
Files already downloaded and verified
cuda:0

10
125
0.001
ResNet34
epoch: 1
Files already downloaded and verified
Files already downloaded and verified
cuda:0
Files already downloaded and verified
Files already downloaded and verified
cuda:0
inter-epoch: 0.125
[1,    50] loss: 1.973
inter-epoch: 0.25
[1,   100] loss: 1.570
inter-epoch: 0.375
[1,   150] loss: 1.399
inter-epoch: 0.5
[1,   200] loss: 1.338
inter-epoch: 0.625
[1,   250] loss: 1.251
inter-epoch: 0.75
[1,   300] loss: 1.234
inter-epoch: 0.875
[1,   350] loss: 1.184
inter-epoch: 1.0
[1,   400] loss: 1.167
Files already downloaded and verified
Files already downloaded and verified
cuda:0
Files already downloaded and verified
Files already downloaded and verified
cuda:0
Valid Loss: 1.177179, accuracy: 0.58
epoch: 2
Files already downloaded and verified
Files already downloaded and verified
cuda:0
Files already downloaded and verified
Files already downloaded and verified
cuda:0
inter-epoch

wandb: Currently logged in as: arahali (use `wandb login --relogin` to force relogin)
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Owner/.netrc
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-03-29 18:36:18.868964: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-29 18:36:18.869536: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
wandb: Tracking run with wandb version 0.12.9
wandb: Syncing run ResNet34-epochs-10
wandb:  View project at https://wandb.ai/arahali/resnet-test
wandb:  View run at https://wandb.ai/arahali/resnet-test/runs/5njmu5nv
wandb: Run data is saved locally in c:\Users\Owner\Documents\Dropbox (M)\PC\Documents\14_Nuvoola\paperspace\wandb\run-20220329_183613-5njmu5nv
wandb: Run `wandb offline` to tu

In [4]:
import wandb
import os

In [5]:
os.environ["WANDB_NOTEBOOK_NAME"] = "./train_model_wandb.ipynb"

# Log in to your W&B account
## Access secrets through SDK?
wandb.login(key='2267b348e1895212f58e3293d96b5e88851beef8')

wandb: Currently logged in as: arahali (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Owner/.netrc


True

In [ ]:
def validate_model(model, valid_dl, loss_func, log_images=False, batch_idx=0):
    "Compute performance of the model on the validation dataset and log a wandb.Table"
    model.eval()
    val_loss = 0.
    with torch.inference_mode():
        correct = 0
        for i, (images, labels) in enumerate(valid_dl, 0):
        # for i, (images, labels) in enumerate(valid_dl), leave=False):
            images, labels = images.to(device), labels.to(device)

            # Forward pass ➡
            outputs = model(images)
            val_loss += loss_func(outputs, labels)*labels.size(0)

            # Compute accuracy and accumulate
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

            # Log one batch of images to the dashboard, always same batch_idx.
            # if i==batch_idx and log_images:
                # log_image_table(images, predicted, labels, outputs.softmax(dim=1))
    return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)

In [8]:
import sys, getopt
import torch.optim as optim
import numpy as np
import torch
import torch.nn as nn
import time

from resnet import resnet18, resnet34
from load_data import trainloader, testloader, valloader
from config.config import model_config


# wandb.init(project="resnet-test")

models = ['ResNet18', 'ResNet34']

for model in models:
    with wandb.init(project="resnet-test", config=model_config, name=model):

        print(model_config['epochs'])
        print(model_config['batch_size'])
        print(model_config['lr'])
        print(model)

        if model == 'ResNet18':
            model = resnet18(3, 10)
        else:
            model = resnet34(3, 10)

        criterion = nn.CrossEntropyLoss()
        # optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
        optimizer = optim.SGD(model.parameters(), lr=model_config['lr'], momentum=0.9)

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model.to(device)

        step = 0
        epoch_durations = []
        for epoch in range(model_config['epochs']):
            
            start_epoch_time = time.time()
            # print("--- %s seconds ---" % (time.time() - start_time))

            print('epoch:', epoch+1)
            mini_batch_check=50
            running_loss = 0.0
            model.train()

            for i, data in enumerate(trainloader, 0):
                # print(i)

                inputs, labels = data[0].to(device), data[1].to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                # print statistics
                running_loss += loss.item()
                
                if i % mini_batch_check == mini_batch_check-1:    # print every 50 mini-batches
                    step +=1
                    print('inter-epoch:', epoch + ((i+1)/len(trainloader)))
                    wandb.log({"train_loss": running_loss/mini_batch_check, "epoch": epoch + ((i+1)/len(trainloader))}, step=step)

                    print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / mini_batch_check))
                    
                    running_loss = 0.0
            
            val_loss, accuracy = validate_model(model, valloader, criterion)
                
            # Log validation metrics
            wandb.log({"val_loss": val_loss, "val_accuracy": accuracy}, step=step)
            print(f"Valid Loss: {val_loss:3f}, accuracy: {accuracy:.2f}")
            epoch_duration = time.time() - start_epoch_time
            wandb.log({"epoch_runtime (seconds)": epoch_duration}, step=step)

            epoch_durations.append(epoch_duration)

        avg_epoch_runtime = sum(epoch_durations) / len(epoch_durations)
        wandb.log({"avg epoch runtime (seconds)": avg_epoch_runtime})
        # wandb.finish()
print('Finished Training')

wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


10
125
0.001
ResNet18
epoch: 1
inter-epoch: 0.125
[1,    50] loss: 1.849
inter-epoch: 0.25
[1,   100] loss: 1.497
inter-epoch: 0.375
[1,   150] loss: 1.401
inter-epoch: 0.5
[1,   200] loss: 1.294
inter-epoch: 0.625
[1,   250] loss: 1.240
inter-epoch: 0.75
[1,   300] loss: 1.210
inter-epoch: 0.875
[1,   350] loss: 1.178
inter-epoch: 1.0
[1,   400] loss: 1.110
Valid Loss: 1.118594, accuracy: 0.59
epoch: 2
inter-epoch: 1.125
[2,    50] loss: 0.950
inter-epoch: 1.25
[2,   100] loss: 0.946
inter-epoch: 1.375
[2,   150] loss: 0.943
inter-epoch: 1.5
[2,   200] loss: 0.953
inter-epoch: 1.625
[2,   250] loss: 0.929
inter-epoch: 1.75
[2,   300] loss: 0.950
inter-epoch: 1.875
[2,   350] loss: 0.895
inter-epoch: 2.0
[2,   400] loss: 0.890
Valid Loss: 1.003573, accuracy: 0.65
epoch: 3
inter-epoch: 2.125
[3,    50] loss: 0.718
inter-epoch: 2.25
[3,   100] loss: 0.711
inter-epoch: 2.375
[3,   150] loss: 0.728
inter-epoch: 2.5
[3,   200] loss: 0.733
inter-epoch: 2.625
[3,   250] loss: 0.759
inter-epoc

avg epoch runtime (seconds),▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch_runtime (seconds),▅▁▂▆█▂▃▃▅▅
train_loss,█▆▆▅▄▄▄▄▃▃▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁
val_accuracy,▁▄▆▇▆▇██▇█
val_loss,▄▂▂▁▃▄▄▅██
avg epoch runtime (seconds),25.17586
epoch,10.0
epoch_runtime (seconds),25.41913
train_loss,0.17155
val_accuracy,0.7115


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


10
125
0.001
ResNet34
epoch: 1
inter-epoch: 0.125
[1,    50] loss: 1.948
inter-epoch: 0.25
[1,   100] loss: 1.593
inter-epoch: 0.375
[1,   150] loss: 1.424
inter-epoch: 0.5
[1,   200] loss: 1.373
inter-epoch: 0.625
[1,   250] loss: 1.251
inter-epoch: 0.75
[1,   300] loss: 1.225
inter-epoch: 0.875
[1,   350] loss: 1.191
inter-epoch: 1.0
[1,   400] loss: 1.168
Valid Loss: 1.188694, accuracy: 0.59
epoch: 2
inter-epoch: 1.125
[2,    50] loss: 0.971
inter-epoch: 1.25
[2,   100] loss: 0.982
inter-epoch: 1.375
[2,   150] loss: 0.965
inter-epoch: 1.5
[2,   200] loss: 0.980
inter-epoch: 1.625
[2,   250] loss: 0.941
inter-epoch: 1.75
[2,   300] loss: 0.951
inter-epoch: 1.875
[2,   350] loss: 0.956
inter-epoch: 2.0
[2,   400] loss: 0.930
Valid Loss: 1.026548, accuracy: 0.63
epoch: 3
inter-epoch: 2.125
[3,    50] loss: 0.723
inter-epoch: 2.25
[3,   100] loss: 0.740
inter-epoch: 2.375
[3,   150] loss: 0.756
inter-epoch: 2.5
[3,   200] loss: 0.777
inter-epoch: 2.625
[3,   250] loss: 0.782
inter-epoc

avg epoch runtime (seconds),▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch_runtime (seconds),█▁▂▂▁▃▃▂▁▁
train_loss,█▆▅▅▄▄▄▄▃▃▄▄▃▃▃▃▂▂▂▃▂▂▂▂▁▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁
val_accuracy,▁▃▆▆▆▇▇▇█▇
val_loss,▆▃▁▂▃▄▅▆▆█
avg epoch runtime (seconds),35.7271
epoch,10.0
epoch_runtime (seconds),35.17793
train_loss,0.20915
val_accuracy,0.6975


Finished Training


In [9]:
for i, data in enumerate(trainloader, 0):
    print(data)
    exit()

[tensor([[[[-0.0039, -0.0118, -0.0039,  ..., -0.0196, -0.0196, -0.0275],
          [-0.0196, -0.0196, -0.0118,  ..., -0.0196, -0.0275, -0.0275],
          [-0.0118, -0.0275, -0.0118,  ..., -0.0275, -0.0353, -0.0353],
          ...,
          [-0.3412, -0.3333, -0.3176,  ..., -0.4902, -0.3804, -0.3098],
          [-0.3255, -0.3490, -0.3490,  ..., -0.4275, -0.3490, -0.3176],
          [-0.3333, -0.3333, -0.3490,  ..., -0.4353, -0.3882, -0.3569]],

         [[ 0.1373,  0.1294,  0.1373,  ...,  0.1137,  0.1137,  0.1059],
          [ 0.1216,  0.1216,  0.1294,  ...,  0.1137,  0.1137,  0.1059],
          [ 0.1373,  0.1216,  0.1373,  ...,  0.1216,  0.1059,  0.1059],
          ...,
          [-0.1843, -0.1843, -0.1686,  ..., -0.3490, -0.2392, -0.1686],
          [-0.1765, -0.2000, -0.2000,  ..., -0.2863, -0.2078, -0.1765],
          [-0.1843, -0.1922, -0.2000,  ..., -0.2941, -0.2471, -0.2157]],

         [[ 0.3098,  0.3020,  0.3098,  ...,  0.2471,  0.2471,  0.2392],
          [ 0.2863,  0.2784, 

In [1]:
len(trainloader)

NameError: name 'trainloader' is not defined

In [ ]:
from load_data import trainloader

Files already downloaded and verified


Files already downloaded and verified


In [ ]:
32*391

12512